In [8]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
from PMLutils import *

# A1

In [3]:
cuda = torch.cuda.is_available()
batch_size = 64
num_epochs = 10

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=1, shuffle=True, **kwargs)



In [7]:
from Models import VAE, convVAE
z = 2
model = VAE(z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")

for epoch in range(1, num_epochs + 1):
    train_VAE(model, optimizer, train_loader, device)
    torch.save(model.state_dict(), "weights/VAE.pth")
    test_VAE(model, device, test_loader)
    with torch.no_grad():
        sample = torch.randn(64, z).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/VAE/sample_' + str(epoch) + '.png')

trainable parameters: 709,588


loss: 15519.8379: 100%|██████████| 469/469 [00:12<00:00, 37.73it/s]


====> Test set loss: 166.8859


loss: 16069.5625: 100%|██████████| 469/469 [00:10<00:00, 46.71it/s]


====> Test set loss: 160.8172


loss: 20168.0000:  28%|██▊       | 132/469 [00:04<00:12, 26.54it/s]


KeyboardInterrupt: 

In [8]:
z = 2
model = convVAE(out_channels = 64, kernel_size = 4, hiddenDim=z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")

trainable parameters: 250,213


In [9]:
for epoch in range(1, num_epochs + 1):
    train_VAE(model, optimizer, test_loader, device)
    torch.save(model.state_dict(), "weights/convVAE.pth")
    test_VAE(model, device, test_loader)
    with torch.no_grad():
        sample = torch.randn(64, z).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/convVAE/sample_' + str(epoch) + '.png')

loss: 194.3179:   2%|▏         | 164/10000 [00:05<05:14, 31.31it/s]


KeyboardInterrupt: 

# A2


Diffusion model

perhaps need to renormalize images for it to work: transforms.Normalize((0.5,),(1.0))

In [11]:
from Diffusion import denoisingDiffusion, deNoise

mps = torch.backends.mps.is_available()
batch_size = 128
num_epochs = 10

torch.manual_seed(1) # args.seed
if mps:
    device = torch.device("mps")
else:
    device = torch.device("cuda" if cuda else "cpu") # args.cuda
print("Using device ", device)

eps_model = deNoise()
eps_model.to(device)
diff = denoisingDiffusion(eps_model, device=device) 
optimizer = torch.optim.Adam(eps_model.parameters(), lr=1e-3)

for epoch in range(1, num_epochs):
    tqdm_ = tqdm(train_loader)
    for data, _ in tqdm_:
        diff.train() # so that everything has gradients and we can do backprop and so on...
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        loss = diff(data)
        loss.backward() # calc gradients
        optimizer.step() # backpropagation
        tqdm_.set_description(f"loss: {loss:.4f}")
    
    diff.eval()
    with torch.no_grad():
        sample = diff.sample_p(n=64)
        save_image(sample.view(64, 1, 28, 28).cpu(),
                    'results/diffusion/sample_' + str(epoch) + '.png')
    torch.save(diff.state_dict(), "weights/diffusion.pth")

Using device  cuda


loss: 0.0176: 100%|██████████| 938/938 [00:37<00:00, 25.34it/s]


KeyboardInterrupt: 

PPCA

In [7]:
from PPCA import PPCA
ppca = PPCA(train_loader.dataset, 20)
samples = ppca.sample()


RuntimeError: torch.linalg.eig: input tensor should not contain infs or NaNs.

Bayesian VAE

In [ ]:
from Models import BayesianVAE

prior = torch.distributions.Normal(torch.tensor([0]), torch.tensor([1]))
model = BayesianVAE(prior)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model.train() # so that everything has gradients and we can do backprop and so on...
train_loss = 0
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.to(device)
    optimizer.zero_grad() # "reset" gradients to 0 for text iteration
    recon_batch, mu, logvar = model(data)
    print(recon_batch)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward() # calc gradients
    train_loss += loss.item()
    optimizer.step() # backpropagation

print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))



model.eval()
test_loss = 0
with torch.no_grad(): # no_grad turns of gradients...
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        test_loss += model.KL_loss().item()

test_loss /= len(test_loader.dataset)
print('====> Test set loss: {:.4f}'.format(test_loss))

Metrics

In [ ]:
def produce_samples(model, input, num_imgs, weights = None, device='cpu'):
    samples = []
    if weights is not None:
        model.load_state_dict(torch.load(weights, map_location=torch.device(device)))
    with torch.no_grad():
        for i in range(num_imgs):
            sample = model.decode(input).cpu()
            samples.append(sample)
    return torch.stack(samples).unsqueeze(1)

In [ ]:
import json
new = False
#Record results
models = [(
            VAE(), "weights/VAE.pth"), (
            convVAE(), "weights/convVAE.pth"), (
            PPCA(train_loader.dataset, 5), None), (
            Diffusion(deNoise(), "weights/diffusion.pth")
            )]
real = []
for i,_ in test_loader: #Get the normalized images
    real.append(i) 
num_imgs = len(real)
real = torch.stack(real).squeeze(1)

if new:
    with open("results/scores.txt", "w") as file:
        file.write("Optimal scores:\n")
        file.write(json.dumps(test_suite(real, real)))
        file.write("\n------------\n")

for m,w in models:
    fake = produce_samples(m, torch.rand(1,2), num_imgs=num_imgs, weights=w)
    scores_dict = test_suite(real, fake)
    with open("results/scores.txt", "a") as file:
        file.write(json.dumps(scores_dict + "\n"))
    